In [7]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [9]:
df_not_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_not_ipcc.jsonl', lines= True)

In [10]:
df_ipcc=df_ipcc[['doi', 'title_OA','topics']]

In [13]:
df_not_ipcc=df_not_ipcc[['doi', 'title_OA','topics']]